In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyomo.environ import *

In [26]:
#Dados de Entrada
agents = ['Cons 1', 'Cons 2', 'Prod A', 'Prod B']
Na = len(agents)
p_max = [100, 50, 100, 80]
p_min = [0, 0, 0, 0]
agent_sign = [1, 1, -1, -1] #1: Consumer; -1: Producer
Ng = sum(1 for p in agent_sign if p == -1)
Nc = sum(1 for p in agent_sign if p == 1)
c_ener = [40, 35, 12, 20]

comunidade = ['Comunidade']
Nco = len(comunidade)

c_ener_exp = [10]
c_ener_imp = [30]


In [27]:
model = ConcreteModel()

#Sets:
model.agent = Set(initialize = agents)
model.com = Set(initialize = comunidade)

#Parâmetros:
model.p_max = Param(model.agent, initialize = {agents[i]: p_max[i] for i in range(Na)})
model.p_min = Param(model.agent, initialize = {agents[i]: p_min[i] for i in range(Na)})
model.agent_sign = Param(model.agent, initialize = {agents[i]: agent_sign[i] for i in range(Na)})
model.c_ener = Param(model.agent, initialize = {agents[i]: c_ener[i] for i in range(Na)})
model.c_ener_imp = Param(model.com, initialize = {comunidade[i]: c_ener_imp[i] for i in range(Nco)})
model.c_ener_exp = Param(model.com, initialize = {comunidade[i]: c_ener_exp[i] for i in range(Nco)})

In [28]:
#Variáveis:
model.p = Var(model.agent, domain=Reals) #Energia total por agente
model.q = Var(model.agent, domain=Reals) #Energia interna

model.beta = Var(model.agent, domain=NonNegativeReals)
model.alpha = Var(model.agent, domain=NonNegativeReals)

model.q_imp = Var(model.com, domain=NonNegativeReals)
model.q_exp = Var(model.com, domain=NonNegativeReals)

In [29]:
def balanco_por_agente_rule(model, agente):
    pn = model.p[agente]
    qn = model.q[agente]
    alphan = model.alpha[agente]
    betan = model.beta[agente]
    return pn + qn - alphan + betan == 0
model.balanco_por_agente = Constraint(model.agent, rule=balanco_por_agente_rule)

def balanco_interno_rule(model):
    return sum(model.q[agente] for agente in model.agent) == 0
model.balanco_interno = Constraint(rule=balanco_interno_rule)

def balanco_importacao_rule(model):
    return sum(model.alpha[agente] for agente in model.agent) == model.q_imp['Comunidade']
model.balanco_importacao = Constraint(rule=balanco_importacao_rule)

def balanco_exportacao_rule(model):
    return sum(model.beta[agente] for agente in model.agent) == model.q_exp['Comunidade']
model.balanco_exportacao = Constraint(rule=balanco_exportacao_rule)

def limites_agentes_rule(model, agente):
    if model.agent_sign[agente] == 1:
        return (model.p_min[agente], model.p[agente], model.p_max[agente])
    else:
        return (-model.p_max[agente], model.p[agente], -model.p_min[agente])
model.limites_agentes = Constraint(model.agent, rule=limites_agentes_rule)

def limite_importacao_rule(model):
    return (model.q_imp['Comunidade']<=sum(model.p[agente]  for agente in model.agent if model.agent_sign[agente] == 1))
model.limite_importacao = Constraint(rule=limite_importacao_rule)

def limite_exporacaio_rule(model):
    return (model.q_exp['Comunidade']<=sum(model.p_max[agente] for agente in model.agent if model.agent_sign[agente]==-1))
model.limite_exportacao = Constraint(rule=limite_exporacaio_rule)

def no_arbitrage_rule(model, agente):
    if model.agent_sign[agente] == 1:
        return model.beta[agente] == 0
    else:
        return model.alpha[agente] == 0
model.no_arbitrage = Constraint(model.agent, rule=no_arbitrage_rule)

In [30]:
#Função Objetivo:
def objective_rule(model):
    CnPn = sum(model.p[agente] * model.c_ener[agente] for agente in model.agent)
    CexpQexp = model.q_exp['Comunidade'] * model.c_ener_exp['Comunidade']
    CimpQimp = model.q_imp['Comunidade'] * model.c_ener_imp['Comunidade']
    return CnPn + CexpQexp - CimpQimp
model.objective = Objective(rule=objective_rule, sense=maximize)

In [31]:
solver = SolverFactory('glpk')
# solver = SolverFactory('ipopt',executable='/content/ipopt')
model.dual = Suffix(direction=Suffix.IMPORT)
resultados = solver.solve(model, tee=False)

In [32]:
# Relatório dos resultados de otimização
print('Status Final do Problema de Otimização:', resultados.solver.status, '\n')
print('Condição de Término:', resultados.solver.termination_condition, '\n')
print('Resultado Função Objetivo:', value(model.objective), '\n')

#Pn:
for agente in model.agent:
    print(f'Energia total do {agente}: {model.p[agente].value}')
print('\n')
#qn:
for agente in model.agent:
    print(f'Energia interna do {agente}: {model.q[agente].value}')
print('\n')

#Alphan:
for agente in model.agent:
    print(f'Alpha do {agente}: {model.alpha[agente].value}')
print('\n')

#Betan
for agente in model.agent:
    print(f'Beta do {agente}: {model.beta[agente].value}')
print('\n')

#Energia importada pela comunidade:
print(f'Energia importada pela comunidade: {model.q_imp["Comunidade"].value}')
print('\n')

#Energia exportada pela comunidade:
print(f'Energia exportada pela comunidade: {model.q_exp["Comunidade"].value}')

Status Final do Problema de Otimização: ok 

Condição de Término: optimal 

Resultado Função Objetivo: 3550.0 

Energia total do Cons 1: 100.0
Energia total do Cons 2: 50.0
Energia total do Prod A: -100.0
Energia total do Prod B: -50.0


Energia interna do Cons 1: -100.0
Energia interna do Cons 2: -50.0
Energia interna do Prod A: 100.0
Energia interna do Prod B: 50.0


Alpha do Cons 1: 0.0
Alpha do Cons 2: 0.0
Alpha do Prod A: 0.0
Alpha do Prod B: 0.0


Beta do Cons 1: 0.0
Beta do Cons 2: 0.0
Beta do Prod A: 0.0
Beta do Prod B: 0.0


Energia importada pela comunidade: 0.0


Energia exportada pela comunidade: -0.0
